<img src="https://i.imgur.com/fOeJmu3.jpg" width="600px">

# Introduction

Hello everyone! In this kernel, I will try to finetune roBERTa base to predict sentiments instead of selected_text. I believe that **explicitly teaching the model to understand the sentiment of a tweet can help it understand which part of the tweet expresses that sentiment**. So, I have decided to undertake a double-finetuning procesdure where I finetune roBERTa base on sentiment classification and then finetune the finetuned model on sentiment extraction (the competition goal). I feel this process might improve the ability of the model to predict selected_text correctly.

This is part one of the approach; I will train the sentiment extraction model in part two :D

## Set up PyTorch-XLA

In [1]:
!curl https://raw.githubusercontent.com/pytorch/xla/master/contrib/scripts/env-setup.py -o pytorch-xla-env-setup.py
!python pytorch-xla-env-setup.py --version nightly --apt-packages libomp5 libopenblas-dev
!export XLA_USE_BF16=1

## Install and import libraries

In [2]:
!pip install -q colored
!pip install -q transformers

In [3]:
import os
import gc
import re

import time
import colored
import numpy as np
import pandas as pd
from colored import fg, bg, attr

import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff
from plotly.subplots import make_subplots

import torch
import torch.nn as nn
from torch.optim import Adam
from torch.utils.data import Dataset, DataLoader

from tqdm.notebook import tqdm
from sklearn.utils import shuffle
from transformers import RobertaModel, RobertaTokenizer

import torch_xla.core.xla_model as xm
import torch_xla.distributed.parallel_loader as pl
import torch_xla.distributed.xla_multiprocessing as xmp

from keras.utils import to_categorical
from keras.preprocessing.sequence import pad_sequences as pad

Using TensorFlow backend.


## Define hyperparameters and load data

In [4]:
EPOCHS = 8
SPLIT = 0.8
MAXLEN = 48
DROP_RATE = 0.225

OUTPUT_UNITS = 3
BATCH_SIZE = 384
LR = (4e-5, 1e-2)
ROBERTA_UNITS = 768
VAL_BATCH_SIZE = 384
MODEL_SAVE_PATH = 'sentiment_model.pt'

np.random.seed(42)
torch.manual_seed(42)
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True

In [5]:
test_df = pd.read_csv('../input/tweet-sentiment-extraction/test.csv')
train_df = pd.read_csv('../input/tweet-sentiment-extraction/train.csv')

In [6]:
test_df.head()

,textID,text,sentiment
0,f87dea47db,Last session of the day http://twitpic.com/67ezh,neutral
1,96d74cb729,Shanghai is also really exciting (precisely -...,positive
2,eee518ae67,"Recession hit Veronique Branquinho, she has to...",negative
3,01082688c6,happy bday!,positive
4,33987a8ee5,http://twitpic.com/4w75p - I like it!!,positive


In [7]:
train_df.head()

,textID,text,selected_text,sentiment
0,cb774db0d1,"I`d have responded, if I were going","I`d have responded, if I were going",neutral
1,549e992a42,Sooo SAD I will miss you here in San Diego!!!,Sooo SAD,negative
2,088c60f138,my boss is bullying me...,bullying me,negative
3,9642c003ef,what interview! leave me alone,leave me alone,negative
4,358bd9e861,"Sons of ****, why couldn`t they put them on t...","Sons of ****,",negative


## Define PyTorch Dataset

In [8]:
class TweetDataset(Dataset):
    def __init__(self, data, tokenizer):
        self.data = data
        self.text = data.text
        self.tokenizer = tokenizer
        self.sentiment = data.sentiment
        self.sentiment_dict = {"positive": 0, "neutral": 1, "negative": 2}

    def __len__(self):
        return len(self.data)

    def __getitem__(self, i):
        pg, tg = 'post', 'post'
        tweet = str(self.text[i]).strip()
        tweet_ids = self.tokenizer.encode(tweet)

        attention_mask_idx = len(tweet_ids) - 1
        if 0 not in tweet_ids: tweet_ids = 0 + tweet_ids
        tweet_ids = pad([tweet_ids], maxlen=MAXLEN, value=1, padding=pg, truncating=tg)

        attention_mask = np.zeros(MAXLEN)
        attention_mask[1:attention_mask_idx] = 1
        attention_mask = attention_mask.reshape((1, -1))
        if 2 not in tweet_ids: tweet_ids[-1], attention_mask[-1] = 2, 0
            
        sentiment = [self.sentiment_dict[self.sentiment[i]]]
        sentiment = torch.FloatTensor(to_categorical(sentiment, num_classes=3))
        return sentiment, torch.LongTensor(tweet_ids), torch.LongTensor(attention_mask)

## Define roBERTa-base model

In [9]:
class Roberta(nn.Module):
    def __init__(self):
        super(Roberta, self).__init__()
        self.softmax = nn.Softmax(dim=1)
        self.drop = nn.Dropout(DROP_RATE)
        self.roberta = RobertaModel.from_pretrained(model)
        self.dense = nn.Linear(ROBERTA_UNITS, OUTPUT_UNITS)
        
    def forward(self, inp, att):
        inp = inp.view(-1, MAXLEN)
        _, self.feat = self.roberta(inp, att)
        return self.softmax(self.dense(self.drop(self.feat)))

## Split data and define tokenizer

In [10]:
train_df = shuffle(train_df)
split = np.int32(SPLIT*len(train_df))
val_df, train_df = train_df[split:], train_df[:split]

In [11]:
model = 'roberta-base'
tokenizer = RobertaTokenizer.from_pretrained(model)

## Define cross entropy and accuracy

In [12]:
def cel(inp, target):
    _, labels = target.max(dim=1)
    return nn.CrossEntropyLoss()(inp, labels)

def accuracy(inp, target):
    inp_ind = inp.max(axis=1).indices
    target_ind = target.max(axis=1).indices
    return (inp_ind == target_ind).float().sum(axis=0)/len(inp_ind)

## Train model on TPU

In [13]:
def print_metric(data, batch, epoch, start, end, metric, typ):

    cs = [211, 212, 213]
    time = np.round(end - start, 1)
    time = "Time: %s{}%s s".format(time)
    fonts = [(fg(c), attr('reset')) for c in cs]
    if typ == "Train": pre = "BATCH %s" + str(batch-1) + "%s  "
    if typ == "Val": pre = "\nEPOCH %s" + str(epoch+1) + "%s  "

    print(pre % fonts[0] , end='')
    t = typ, metric, "%s", data, "%s"
    print("{} {}: {}{}{}".format(*t) % fonts[1] + "  " + time % fonts[2])

In [14]:
device = xm.xla_device()

val_df = val_df.reset_index(drop=True)
val_set = TweetDataset(val_df, tokenizer)
val_loader = DataLoader(val_set, batch_size=VAL_BATCH_SIZE)

train_df = train_df.reset_index(drop=True)
train_set = TweetDataset(train_df, tokenizer)
train_loader = DataLoader(train_set, batch_size=BATCH_SIZE, shuffle=True)
    
network = Roberta().to(device)
optimizer = Adam([{'params': network.dense.parameters(), 'lr': LR[1]},
                  {'params': network.roberta.parameters(), 'lr': LR[0]}])

val_losses, val_accuracies = [], []
train_losses, train_accuracies = [], []
    
start = time.time()
print("STARTING TRAINING ...\n")

for epoch in range(EPOCHS):

    batch = 1
    network.train()
    fonts = (fg(48), attr('reset'))
    print(("EPOCH %s" + str(epoch+1) + "%s") % fonts)
        
    for train_batch in train_loader:
        train_targ, train_in, train_att = train_batch
            
        network = network.to(device)
        train_in = train_in.to(device)
        train_att = train_att.to(device)
        train_targ = train_targ.to(device)

        train_preds = network.forward(train_in, train_att)
        train_loss = cel(train_preds, train_targ.squeeze(dim=1))
        train_accuracy = accuracy(train_preds, train_targ.squeeze(dim=1))

        optimizer.zero_grad()
        train_loss.backward()
        xm.optimizer_step(optimizer, barrier=True)
            
        end = time.time()
        batch = batch + 1
        acc = np.round(train_accuracy.item(), 3)
        print_metric(acc, batch, None, start, end, metric="acc", typ="Train")

    network.eval()
    val_loss, val_accuracy = 0, 0
    for val_batch in tqdm(val_loader):
        targ, val_in, val_att = val_batch

        with torch.no_grad():
            targ = targ.to(device)
            val_in = val_in.to(device)
            val_att = val_att.to(device)
            network = network.to(device)

            pred = network.forward(val_in, val_att)
            val_loss += cel(pred, targ.squeeze(dim=1)).item()*len(pred)
            val_accuracy += accuracy(pred, targ.squeeze(dim=1)).item()*len(pred)
        
    end = time.time()
    val_loss /= len(val_set)
    val_accuracy /= len(val_set)
    acc = np.round(val_accuracy, 3)
    print_metric(acc, None, epoch, start, end, metric="acc", typ="Val")
    
    print("")
    val_losses.append(val_loss); train_losses.append(train_loss)
    val_accuracies.append(val_accuracy); train_accuracies.append(train_accuracy)
    
print("ENDING TRAINING ...")


STARTING TRAINING ...

EPOCH 1
BATCH 1  Train acc: 0.299  Time: 44.3 s
BATCH 2  Train acc: 0.372  Time: 92.5 s
BATCH 3  Train acc: 0.409  Time: 93.5 s
BATCH 4  Train acc: 0.427  Time: 94.5 s
BATCH 5  Train acc: 0.391  Time: 95.5 s
BATCH 6  Train acc: 0.396  Time: 96.4 s
BATCH 7  Train acc: 0.401  Time: 97.4 s
BATCH 8  Train acc: 0.404  Time: 98.4 s
BATCH 9  Train acc: 0.359  Time: 99.3 s
BATCH 10  Train acc: 0.404  Time: 100.3 s
BATCH 11  Train acc: 0.359  Time: 101.3 s
BATCH 12  Train acc: 0.388  Time: 102.3 s
BATCH 13  Train acc: 0.352  Time: 103.2 s
BATCH 14  Train acc: 0.328  Time: 104.2 s
BATCH 15  Train acc: 0.417  Time: 105.1 s
BATCH 16  Train acc: 0.414  Time: 106.0 s
BATCH 17  Train acc: 0.409  Time: 107.0 s
BATCH 18  Train acc: 0.438  Time: 107.9 s
BATCH 19  Train acc: 0.417  Time: 108.9 s
BATCH 20  Train acc: 0.398  Time: 109.8 s
BATCH 21  Train acc: 0.375  Time: 110.7 s
BATCH 22  Train acc: 0.346  Time: 111.7 s
BATCH 23  Train acc: 0.414  Time: 112.6 s
BATCH 24  Train acc:



EPOCH 1  Val acc: 0.687  Time: 221.0 s

EPOCH 2
BATCH 1  Train acc: 0.677  Time: 290.3 s
BATCH 2  Train acc: 0.646  Time: 291.3 s
BATCH 3  Train acc: 0.685  Time: 292.2 s
BATCH 4  Train acc: 0.701  Time: 293.1 s
BATCH 5  Train acc: 0.628  Time: 294.0 s
BATCH 6  Train acc: 0.68  Time: 294.9 s
BATCH 7  Train acc: 0.708  Time: 295.8 s
BATCH 8  Train acc: 0.648  Time: 296.7 s
BATCH 9  Train acc: 0.555  Time: 297.6 s
BATCH 10  Train acc: 0.578  Time: 298.4 s
BATCH 11  Train acc: 0.677  Time: 299.3 s
BATCH 12  Train acc: 0.711  Time: 300.2 s
BATCH 13  Train acc: 0.667  Time: 301.1 s
BATCH 14  Train acc: 0.68  Time: 302.0 s
BATCH 15  Train acc: 0.63  Time: 302.9 s
BATCH 16  Train acc: 0.682  Time: 303.8 s
BATCH 17  Train acc: 0.711  Time: 304.7 s
BATCH 18  Train acc: 0.695  Time: 305.6 s
BATCH 19  Train acc: 0.714  Time: 306.5 s
BATCH 20  Train acc: 0.677  Time: 307.4 s
BATCH 21  Train acc: 0.656  Time: 308.3 s
BATCH 22  Train acc: 0.695  Time: 309.2 s
BATCH 23  Train acc: 0.672  Time: 310.



EPOCH 2  Val acc: 0.732  Time: 349.6 s

EPOCH 3
BATCH 1  Train acc: 0.745  Time: 349.9 s
BATCH 2  Train acc: 0.747  Time: 350.9 s
BATCH 3  Train acc: 0.771  Time: 351.8 s
BATCH 4  Train acc: 0.74  Time: 352.7 s
BATCH 5  Train acc: 0.693  Time: 353.6 s
BATCH 6  Train acc: 0.701  Time: 354.5 s
BATCH 7  Train acc: 0.727  Time: 355.3 s
BATCH 8  Train acc: 0.706  Time: 356.2 s
BATCH 9  Train acc: 0.711  Time: 357.1 s
BATCH 10  Train acc: 0.753  Time: 358.0 s
BATCH 11  Train acc: 0.714  Time: 358.9 s
BATCH 12  Train acc: 0.771  Time: 359.8 s
BATCH 13  Train acc: 0.755  Time: 360.7 s
BATCH 14  Train acc: 0.745  Time: 361.6 s
BATCH 15  Train acc: 0.734  Time: 362.5 s
BATCH 16  Train acc: 0.786  Time: 363.4 s
BATCH 17  Train acc: 0.745  Time: 364.3 s
BATCH 18  Train acc: 0.703  Time: 365.2 s
BATCH 19  Train acc: 0.76  Time: 366.1 s
BATCH 20  Train acc: 0.773  Time: 367.0 s
BATCH 21  Train acc: 0.747  Time: 367.8 s
BATCH 22  Train acc: 0.742  Time: 368.7 s
BATCH 23  Train acc: 0.755  Time: 369



EPOCH 3  Val acc: 0.762  Time: 409.0 s

EPOCH 4
BATCH 1  Train acc: 0.766  Time: 409.3 s
BATCH 2  Train acc: 0.768  Time: 410.2 s
BATCH 3  Train acc: 0.784  Time: 411.1 s
BATCH 4  Train acc: 0.758  Time: 412.0 s
BATCH 5  Train acc: 0.792  Time: 412.9 s
BATCH 6  Train acc: 0.74  Time: 413.8 s
BATCH 7  Train acc: 0.781  Time: 414.7 s
BATCH 8  Train acc: 0.784  Time: 415.6 s
BATCH 9  Train acc: 0.76  Time: 416.5 s
BATCH 10  Train acc: 0.773  Time: 417.3 s
BATCH 11  Train acc: 0.773  Time: 418.2 s
BATCH 12  Train acc: 0.747  Time: 419.1 s
BATCH 13  Train acc: 0.797  Time: 420.0 s
BATCH 14  Train acc: 0.763  Time: 420.9 s
BATCH 15  Train acc: 0.753  Time: 421.8 s
BATCH 16  Train acc: 0.781  Time: 422.7 s
BATCH 17  Train acc: 0.758  Time: 423.6 s
BATCH 18  Train acc: 0.76  Time: 424.5 s
BATCH 19  Train acc: 0.802  Time: 425.3 s
BATCH 20  Train acc: 0.747  Time: 426.2 s
BATCH 21  Train acc: 0.807  Time: 427.1 s
BATCH 22  Train acc: 0.784  Time: 428.0 s
BATCH 23  Train acc: 0.763  Time: 428.



EPOCH 4  Val acc: 0.771  Time: 468.1 s

EPOCH 5
BATCH 1  Train acc: 0.812  Time: 468.4 s
BATCH 2  Train acc: 0.74  Time: 469.4 s
BATCH 3  Train acc: 0.81  Time: 470.3 s
BATCH 4  Train acc: 0.768  Time: 471.2 s
BATCH 5  Train acc: 0.797  Time: 472.1 s
BATCH 6  Train acc: 0.807  Time: 472.9 s
BATCH 7  Train acc: 0.807  Time: 473.8 s
BATCH 8  Train acc: 0.789  Time: 474.7 s
BATCH 9  Train acc: 0.781  Time: 475.6 s
BATCH 10  Train acc: 0.797  Time: 476.5 s
BATCH 11  Train acc: 0.773  Time: 477.4 s
BATCH 12  Train acc: 0.786  Time: 478.3 s
BATCH 13  Train acc: 0.802  Time: 479.2 s
BATCH 14  Train acc: 0.779  Time: 480.1 s
BATCH 15  Train acc: 0.763  Time: 481.0 s
BATCH 16  Train acc: 0.81  Time: 481.9 s
BATCH 17  Train acc: 0.76  Time: 482.8 s
BATCH 18  Train acc: 0.794  Time: 483.6 s
BATCH 19  Train acc: 0.766  Time: 484.5 s
BATCH 20  Train acc: 0.805  Time: 485.4 s
BATCH 21  Train acc: 0.768  Time: 486.3 s
BATCH 22  Train acc: 0.815  Time: 487.2 s
BATCH 23  Train acc: 0.766  Time: 488.1



EPOCH 5  Val acc: 0.772  Time: 527.3 s

EPOCH 6
BATCH 1  Train acc: 0.797  Time: 527.6 s
BATCH 2  Train acc: 0.812  Time: 528.5 s
BATCH 3  Train acc: 0.833  Time: 529.4 s
BATCH 4  Train acc: 0.805  Time: 530.3 s
BATCH 5  Train acc: 0.792  Time: 531.2 s
BATCH 6  Train acc: 0.812  Time: 532.1 s
BATCH 7  Train acc: 0.807  Time: 533.0 s
BATCH 8  Train acc: 0.771  Time: 533.9 s
BATCH 9  Train acc: 0.781  Time: 534.8 s
BATCH 10  Train acc: 0.766  Time: 535.6 s
BATCH 11  Train acc: 0.758  Time: 536.5 s
BATCH 12  Train acc: 0.773  Time: 537.4 s
BATCH 13  Train acc: 0.74  Time: 538.3 s
BATCH 14  Train acc: 0.784  Time: 539.2 s
BATCH 15  Train acc: 0.771  Time: 540.1 s
BATCH 16  Train acc: 0.776  Time: 541.0 s
BATCH 17  Train acc: 0.81  Time: 541.9 s
BATCH 18  Train acc: 0.789  Time: 542.7 s
BATCH 19  Train acc: 0.76  Time: 543.6 s
BATCH 20  Train acc: 0.81  Time: 544.5 s
BATCH 21  Train acc: 0.737  Time: 545.4 s
BATCH 22  Train acc: 0.781  Time: 546.3 s
BATCH 23  Train acc: 0.805  Time: 547.2



EPOCH 6  Val acc: 0.775  Time: 586.4 s

EPOCH 7
BATCH 1  Train acc: 0.797  Time: 586.7 s
BATCH 2  Train acc: 0.818  Time: 587.7 s
BATCH 3  Train acc: 0.797  Time: 588.6 s
BATCH 4  Train acc: 0.807  Time: 589.5 s
BATCH 5  Train acc: 0.812  Time: 590.4 s
BATCH 6  Train acc: 0.81  Time: 591.3 s
BATCH 7  Train acc: 0.799  Time: 592.2 s
BATCH 8  Train acc: 0.797  Time: 593.1 s
BATCH 9  Train acc: 0.745  Time: 594.0 s
BATCH 10  Train acc: 0.784  Time: 594.9 s
BATCH 11  Train acc: 0.771  Time: 595.7 s
BATCH 12  Train acc: 0.82  Time: 596.6 s
BATCH 13  Train acc: 0.781  Time: 597.5 s
BATCH 14  Train acc: 0.779  Time: 598.4 s
BATCH 15  Train acc: 0.81  Time: 599.3 s
BATCH 16  Train acc: 0.792  Time: 600.2 s
BATCH 17  Train acc: 0.815  Time: 601.1 s
BATCH 18  Train acc: 0.812  Time: 602.0 s
BATCH 19  Train acc: 0.82  Time: 602.9 s
BATCH 20  Train acc: 0.789  Time: 603.8 s
BATCH 21  Train acc: 0.784  Time: 604.7 s
BATCH 22  Train acc: 0.794  Time: 605.6 s
BATCH 23  Train acc: 0.755  Time: 606.5



EPOCH 7  Val acc: 0.783  Time: 645.7 s

EPOCH 8
BATCH 1  Train acc: 0.755  Time: 646.0 s
BATCH 2  Train acc: 0.833  Time: 646.9 s
BATCH 3  Train acc: 0.802  Time: 647.8 s
BATCH 4  Train acc: 0.779  Time: 648.7 s
BATCH 5  Train acc: 0.805  Time: 649.6 s
BATCH 6  Train acc: 0.766  Time: 650.5 s
BATCH 7  Train acc: 0.805  Time: 651.4 s
BATCH 8  Train acc: 0.789  Time: 652.3 s
BATCH 9  Train acc: 0.818  Time: 653.2 s
BATCH 10  Train acc: 0.789  Time: 654.1 s
BATCH 11  Train acc: 0.797  Time: 654.9 s
BATCH 12  Train acc: 0.805  Time: 655.8 s
BATCH 13  Train acc: 0.797  Time: 656.7 s
BATCH 14  Train acc: 0.841  Time: 657.6 s
BATCH 15  Train acc: 0.807  Time: 658.5 s
BATCH 16  Train acc: 0.831  Time: 659.4 s
BATCH 17  Train acc: 0.81  Time: 660.3 s
BATCH 18  Train acc: 0.805  Time: 661.2 s
BATCH 19  Train acc: 0.828  Time: 662.1 s
BATCH 20  Train acc: 0.82  Time: 663.0 s
BATCH 21  Train acc: 0.779  Time: 663.8 s
BATCH 22  Train acc: 0.786  Time: 664.7 s
BATCH 23  Train acc: 0.805  Time: 665



EPOCH 8  Val acc: 0.781  Time: 704.8 s

ENDING TRAINING ...


## Check testing performance

In [17]:
test_set = TweetDataset(test_df, tokenizer)
test_loader = DataLoader(test_set, batch_size=VAL_BATCH_SIZE)

network.eval()
for test_batch in tqdm(test_loader):
    test_preds, test_targs = [], []
    targ, test_in, test_att = test_batch

    with torch.no_grad():
        network = network.to(device)
        test_in = test_in.to(device)
        test_att = test_att.to(device)
        pred = network.forward(test_in, test_att)
        test_preds.append(pred); test_targs.append(targ)

test_preds = torch.cat(test_preds, axis=0)
test_targs = torch.cat(test_targs, axis=0)
test_accuracy = accuracy(test_preds, test_targs.squeeze(dim=1).to(device))

In [18]:
fonts = (fg(212), attr('reset'))
acc = np.round(test_accuracy.item()*100, 2)
print("{}: {}{}{}".format("Test acc", "%s", str(acc), "%s") % fonts + " %")

Test acc: 91.03 %


## Sample sentiment prediction

In [19]:
def predict_sentiment(tweet):
    pg, tg = 'post', 'post'
    tweet_ids = tokenizer.encode(tweet.strip())
    sent = {0: 'positive', 1: 'neutral', 2: 'negative'}

    att_mask_idx = len(tweet_ids) - 1
    if 0 not in tweet_ids: tweet_ids = 0 + tweet_ids
    tweet_ids = pad([tweet_ids], maxlen=MAXLEN, value=1, padding=pg, truncating=tg)

    att_mask = np.zeros(MAXLEN)
    att_mask[1:att_mask_idx] = 1
    att_mask = att_mask.reshape((1, -1))
    if 2 not in tweet_ids: tweet_ids[-1], att_mask[-1] = 2, 0
    tweet_ids, att_mask = torch.LongTensor(tweet_ids), torch.LongTensor(att_mask)
    return sent[np.argmax(network.forward(tweet_ids.to(device), att_mask.to(device)).detach().cpu().numpy())]

In [20]:
predict_sentiment("It was okay I guess?")

'neutral'

In [21]:
predict_sentiment("I want to hide omg !!!")

'neutral'

In [22]:
predict_sentiment("I am feeling great today ...")

'positive'

## Save model for PART 2

In [23]:
network = network.cpu()
torch.save(network.state_dict(), MODEL_SAVE_PATH)